We want to isolate households with two parents living under civil union with children being over (or not) 21/25 yo.

In [ ]:
import datetime
start_time = datetime.datetime.now(); print(start_time)

In [ ]:
secret = False #Will be put to True for a CASD export

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd ; pd.set_option("display.max_columns",200)
import pickle

In [ ]:
pickle_year_variable_path = "C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Program/year_variable/"
year =  pickle.load(open(pickle_year_variable_path+"year_variable.p",'rb')); print(year)

In [ ]:
store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/edp_2016_final.h5'
reaction_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/reaction.h5'
pickles_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/pickles/{}'.format(year)
Store = pd.HDFStore(store_path)

In [ ]:
Store

In [ ]:
for key in Store.keys(): print(key)
Store.close()

In [ ]:
fisc_revdet = pd.read_hdf( store_path,"FISC_REVDET_{}".format(year+1) )
fisc_revenu = pd.read_hdf(store_path,"FISC_REVENU_{}".format(year+1))
fisc_individu = pd.read_hdf( store_path,"FISC_INDIVIDU_{}".format(year+1) )

In [ ]:
year

In [ ]:
print(fisc_individu.shape, fisc_revenu.shape, fisc_individu.shape)

In [ ]:
assert np.all(fisc_revenu.AN_REVENU == year)

## Data cleaning

All operations needed to process raw data in a good way

In [ ]:
fisc_individu["CIDECI"]= fisc_individu.CIDECI.astype("str")

In [ ]:
fisc_revenu["ID_FISC_LOG_DIFF"] = fisc_revenu.ID_FISC_LOG_DIFF.astype("float")

In [ ]:
fisc_revenu.sort_values("ID_FISC_LOG_DIFF").head()

In [ ]:
(fisc_revenu.NBPERSM - fisc_revenu.NBPAC).value_counts() # 2, soit couples marié, soit célib avec enfant majeur, etc.

### Drop duplicates

On droppe s'il y a plus d'un IDFISC_LOG_DIFF dans fisc_revenu

In [ ]:
#Test qu'il n'y a pas de duplicate dans la base
assert (fisc_revenu.ID_FISC_LOG_DIFF.duplicated()).value_counts().index == False

In [ ]:
assert(fisc_individu[["ID_FISC_FOY_DIFF", "ORDREFIP", "TYPE_FISC"]
                    ].duplicated()).value_counts().index == False

### Drop not same number of individuals

- Compte le nombre d'individus par logement fiscal dans la table fiscindividu  
- Regarde le nombre de personne dans le logement fiscal (variable NBFOYM)  
- Droppe les logement fiscaux ou le nombre n'est pas exactement le même.

In [ ]:
%%time
#compte
nb_pers_by_id_fisclog_in_fisci = fisc_individu.groupby("ID_FISC_LOG_DIFF")["ANAIS"].count();
nb_pers_by_id_fisclog_in_fisci.name = "Nbpers_in_fisci_by_fisclog"
#compare
merge = pd.merge(nb_pers_by_id_fisclog_in_fisci.to_frame().reset_index(),
                 fisc_revenu[["ID_FISC_LOG_DIFF", "NBPERSM", "NBFOYM", "NBPAC"]],
                 on = "ID_FISC_LOG_DIFF" )
print(
    (merge.NBPERSM == merge.Nbpers_in_fisci_by_fisclog).value_counts())
id_fisc_log_to_keep = merge[merge.NBPERSM == merge.Nbpers_in_fisci_by_fisclog].ID_FISC_LOG_DIFF


In [ ]:
#fisc_revdet = fisc_revdet[~ fisc_revdet.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_drop)]
fisc_revenu = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_keep)]
fisc_individu = fisc_individu[ fisc_individu.ID_FISC_LOG_DIFF.isin(id_fisc_log_to_keep)]

In [ ]:
print(fisc_revenu.shape, fisc_individu.shape)

## Obtenir la structure familiale

In [ ]:
%%time
grpby_fisci = fisc_individu.groupby('ID_FISC_LOG_DIFF').CIDECI.apply("".join)

In [ ]:
grpby_fisci.value_counts().head(10)

Ceci nous donne une vague idée de la structure familiale.  
"MM" représente deux individus mariés, chaque "A" représente une personne à charge.  
"MMAA" représente donc un logement avec deux individus mariés avec deux individus à charge.  
"MMC" représente un logement d'un couple marié avec un célibataire.


"C" représente les logement fiscaux de célibataires, "CA" célib avec un enfant, etc.



Cela pourrait ne pas être un couple marié avec deux enfants. Cela pourrait être une mêre marié avec son enfant également marié mais dont l'épouse vivrait sous un autre toit, avec sa mère dépendente à charge, et son marri qui vivrait ailleur dans le logement (e.g. en instance de divorce).

Bien sur dans la majorité des cas cela est effectivement un couple mariés avec deux enfants...  

### Preuve :

#### Un seul foyer fiscal

In [ ]:
df = pd.DataFrame()
df["MMAA"] = (grpby_fisci == "MMAA")
df["CIDECI"] = grpby_fisci.values
MMAA_id_fisc_log_diff = df[df.MMAA == True].index
fisc_revenu_MMAA = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(MMAA_id_fisc_log_diff)]
fisc_revenu_MMAA.NBFOYM.value_counts(normalize = True).head(3)

99.9% ont un seul foyer fiscal dans le logement ce qui correspondrait effectivement à un couple marié, 0.13% en on deux ou plus.

Qui sont donc ces 0.13% de MMAA avec plus d'1 foyer fiscal ?

In [ ]:
MMAA_id_fisc_log_diff_2_fiscfoy = fisc_revenu_MMAA.query("NBFOYM>1").ID_FISC_LOG_DIFF

In [ ]:
MMAA_id_fisc_log_diff_2_fiscfoy.shape

Ils sont 367, on va faire comme s'ils n'existaient pas...

In [ ]:
# if secret == False : 
#     display(
#         fisc_individu[
#         fisc_individu.ID_FISC_LOG_DIFF.isin(MMAA_id_fisc_log_diff_2_fiscfoy)
#         ].sort_values("ID_FISC_FOY_DIFF").head(10)[["ID_FISC_FOY_DIFF","ID_FISC_LOG_DIFF","CIDECI",
#                                                     "SEXE","ANAIS", "TYPE_FISC"]]
#     )#Voila tout de même un moyen de les voir

On revient sur MMAA

In [ ]:
MMAA_id_fisc_log_diff_1_fiscfoy = fisc_revenu_MMAA.query("NBFOYM==1").ID_FISC_LOG_DIFF

### Condition d'age

C'est bien deux enfants et deux parents.

In [ ]:
MMAA_id_fisc_log_diff_1_fiscfoy = fisc_revenu_MMAA.query("NBFOYM==1").ID_FISC_LOG_DIFF

In [ ]:
fisc_individu_MMAA_1_fisc_foy = fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(MMAA_id_fisc_log_diff_1_fiscfoy)
                                ].sort_values("ID_FISC_FOY_DIFF")

In [ ]:
if secret == False : 
    display(
        fisc_individu_MMAA_1_fisc_foy.head(10)[["ID_FISC_FOY_DIFF","ID_FISC_LOG_DIFF","CIDECI",
                                                    "SEXE","ANAIS", "TYPE_FISC"]]
    )

On va bien vérifier que les déclarants (CIDECI == 1 ou 2), sont plus vieux que les personnes à charges.  

Algorithme: On prend le plus jeune des déclarants, et on vérifie qu'il est plus vieux que le plus vieux des personnes à charge.



In [ ]:
younger_tax_filler = fisc_individu_MMAA_1_fisc_foy.query(
    "((TYPE_FISC=='1')|(TYPE_FISC=='2'))").groupby("ID_FISC_LOG_DIFF").ANAIS.max()                                                
oldest_dependent = fisc_individu_MMAA_1_fisc_foy[
                        fisc_individu_MMAA_1_fisc_foy.TYPE_FISC.isin(
                            ["A", "B","C", "D", "E", "F", "G", "H","I", "J",
                            "3", "4", "5", "6", "7", "8", "9"]
                            )               
                                ].groupby("ID_FISC_LOG_DIFF").ANAIS.max()                                                

In [ ]:
younger_tax_filler.name = "ANAIS_yrgr_tf"
oldest_dependent.name = "ANAIS_oldest_dependent"

In [ ]:
df_youg_tf_old_depend = pd.concat([younger_tax_filler, oldest_dependent], axis = 1)

In [ ]:
((df_youg_tf_old_depend.ANAIS_oldest_dependent)< (df_youg_tf_old_depend.ANAIS_yrgr_tf)).value_counts()

On a que 3832 logements fiscaux MMAA avec une pesonne dépendente plus vielle.
Il est possible de mettre sur sa feuille fiscale les parents dépendents... (qui seraient donc soit "A" soit "3" dans la variable TYPE_FISC)

In [ ]:

df_youg_tf_old_depend[((df_youg_tf_old_depend.ANAIS_oldest_dependent)
                       < (df_youg_tf_old_depend.ANAIS_yrgr_tf))].shape

On vérifie en regardant sur l'ensemble de la population.

In [ ]:
younger_tax_filler = fisc_individu.query(
    "((TYPE_FISC=='1')|(TYPE_FISC=='2'))").groupby("ID_FISC_LOG_DIFF").ANAIS.max()                                                
oldest_dependent = fisc_individu[
                        fisc_individu.TYPE_FISC.isin(
                            ["A", "B","C", "D", "E", "F", "G", "H","I", "J",
                            "3", "4", "5", "6", "7", "8", "9"]
                            )               
                                ].groupby("ID_FISC_LOG_DIFF").ANAIS.max()                                                

In [ ]:
younger_tax_filler.name = "ANAIS_yrgr_tf"
oldest_dependent.name = "ANAIS_oldest_dependent"
df_youg_tf_old_depend = pd.concat([younger_tax_filler, oldest_dependent], axis = 1)
(df_youg_tf_old_depend.ANAIS_oldest_dependent< df_youg_tf_old_depend.ANAIS_yrgr_tf).value_counts()

In [ ]:
fisc_individu.shape

In [ ]:
df_youg_tf_old_depend[
    (df_youg_tf_old_depend.ANAIS_oldest_dependent< df_youg_tf_old_depend.ANAIS_yrgr_tf)].head(4)

Ici, il semble que les personnes dépendentes ont plutôt tendence à être jeunes... plutot que des grands parents rattachés.

### MMC va donc majoritairement représenter les ménages avec un majeur non rataché
#### Preuve:

In [ ]:
df = pd.DataFrame()
df["MMC"] = (grpby_fisci == "MMC")
df["CIDECI"] = grpby_fisci.values
MMC_id_fisc_log_diff = df[df.MMC == True].index
fisc_revenu_MMC = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(MMC_id_fisc_log_diff)]
fisc_revenu_MMC.NBFOYM.value_counts(normalize = True).head(3)

 Il y a deux foyers fiscaux

In [ ]:
MMC_id_fisc_log_diff_2_fiscfoy = fisc_revenu_MMC.query("NBFOYM==2").ID_FISC_LOG_DIFF
fisc_individu_MMC_2_fisc_foy = fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(MMC_id_fisc_log_diff_2_fiscfoy)
                                ].sort_values("ID_FISC_LOG_DIFF")

In [ ]:
if secret == False : 
    display(
        fisc_individu_MMC_2_fisc_foy.head(10)[["ID_FISC_FOY_DIFF","ID_FISC_LOG_DIFF","CIDECI",
                                                    "SEXE","ANAIS", "TYPE_FISC"]]
    )

On voit en regardants l'age que c'est dans la grande majorité des cas un enfant vivant avec ses parents.
TODO: le montrer

### Isoler les groupes d'intérêt en fonction de la structure familiale:

1. Couple mariés ou pacés sans enfants:  
Conditions: (grpby_CIDECI = "MM" ou "OO")et (NBFOYM = 1) et NBPERSM ==2
2. Couple mariés ou pacés avec enfants:
   1. Sans majeurs:  
      Conditions: 
          ((grpby_CIDECI where TYPEFISC==(1|2)) = "MM" ou "OO")  
          ((grpby_CIDECI where TYPEFISC==("A",..., "3")) = "only_letters")
              control : (NBFOYM == 1)
   2. Avec majeurs ratachés:
           ((grpby_CIDECI where TYPEFISC==(1|2)) = "MM" ou "OO")  
               control = (NBFOYM == 1) (on peut potentiellement avoir quelques nouveaux couples en cours 
                                             de     séparation.)
                                          
   3. Avec majeurs non ratachés (n'ayant pas d'enfants) + point 2 ou 3:  
           ((grpby_CIDECI where TYPEFISC==(2)) = "M" ou "O") #un seul déclarant 2  
           info: NPPERSM - NBPAC == 2 + nombre de majeurs non ratachés.
           control: "C" number == nombre de majeurs non ratachés
           
   
3. Célibtaire sans enfants:
Conditions : NBPERM == 1 et  (grpby_CIDECI == "C"|"D")  #On élimine les divorcés sans enfants  

4. Célibataire avec enfants:  
     1. Conditions: Sans majeurs:
     Conditions: (grpby_CIDECI where TYPEFISC==(1)) == "C" ou "D")
                 ((grpby_CIDECI where TYPEFISC==("A",..., "3")) = "only_letters")
                      control : (NBFOYM == 1)
                      
etc                  
                                             
        
          
   

In [ ]:
%%time
declrants_groupby_CIDECI = fisc_individu[
                                fisc_individu.TYPE_FISC.isin(["1", "2"])
                                        ].groupby('ID_FISC_LOG_DIFF').CIDECI.apply("".join)

In [ ]:
(declrants_groupby_CIDECI == "MM").value_counts()

In [ ]:
id_fisc_log_declarants_MM_OO = declrants_groupby_CIDECI[
                    ((declrants_groupby_CIDECI == "MM")|(declrants_groupby_CIDECI == "MM"))
                                                        ].index
id_fisc_log_declarants_C = declrants_groupby_CIDECI[
                    ((declrants_groupby_CIDECI == "C"))].index

In [ ]:
#pd.value_counts().__defaults__  = (None, True, False, False, None, False)

In [ ]:
%%time
declrant_2_groupby_CIDECI = fisc_individu[
                                fisc_individu.TYPE_FISC.isin(["2"])
                                        ].groupby('ID_FISC_LOG_DIFF').CIDECI.apply("".join)

In [ ]:
print(declrant_2_groupby_CIDECI.value_counts(dropna = False).head())

In [ ]:
filter_unique_declarant_2 = (declrant_2_groupby_CIDECI.str.len()==1)

In [ ]:
## Test if CIDECI == M | O
print(declrant_2_groupby_CIDECI[filter_unique_declarant_2].value_counts())
#TODO: print % de non M|O
print((declrant_2_groupby_CIDECI[filter_unique_declarant_2].isin(["M", "O"])).value_counts(normalize = True))

In [ ]:
### On vire les Divorcés veuf et célibs

In [ ]:
unique_declrant_2_groupby_CIDECI = declrant_2_groupby_CIDECI[filter_unique_declarant_2]

In [ ]:
filter_only_declar_2_with_O_or_M = unique_declrant_2_groupby_CIDECI.isin(["M", "O"])
unique_declrant_2_groupby_CIDECI_O_or_M = unique_declrant_2_groupby_CIDECI[filter_only_declar_2_with_O_or_M]

In [ ]:
id_fisc_log_unique_declrant_2_grpby_CIDECI_O_or_M = unique_declrant_2_groupby_CIDECI_O_or_M.index

In [ ]:
len(id_fisc_log_unique_declrant_2_grpby_CIDECI_O_or_M)

In [ ]:
dependent_person_type_fisc_label = ["A", "B","C", "D", "E", "F", "G", "H","I", "J",
                                "3", "4", "5", "6", "7", "8", "9"]

In [ ]:
%%time
dependent_groupby_TYPE_FISC = fisc_individu[
                                fisc_individu.TYPE_FISC.isin(dependent_person_type_fisc_label)
                                        ].groupby('ID_FISC_LOG_DIFF').TYPE_FISC.apply("".join)

In [ ]:
df = pd.DataFrame()
df["TYPE_FISC"] = dependent_groupby_TYPE_FISC
df["Majeurs_rataches"] = dependent_groupby_TYPE_FISC.str.contains("[1-9]", regex=True) #Regex contains a number
df["Enf_ss_majeur_rataches"] = ~dependent_groupby_TYPE_FISC.str.contains("[1-9]", regex=True)
id_fisc_log_majeurs_rataches = df[df.Majeurs_rataches].index
id_fisc_log_enf_ss_majeurs_rataches = df[df.Enf_ss_majeur_rataches].index

In [ ]:
%%time
##### Majeurs non ratachés
# Only one conjoint
declrant2_groupby_CIDECI = fisc_individu[  
                                fisc_individu.TYPE_FISC.isin(["2"])
                                        ].groupby('ID_FISC_LOG_DIFF').CIDECI.apply("".join)

In [ ]:
more_than_one_joint_tax_fiscal_household = declrant2_groupby_CIDECI[declrant2_groupby_CIDECI.str.len()>1]
id_more_than_one_joint_tax_fiscal_household = more_than_one_joint_tax_fiscal_household.index

In [ ]:
fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_more_than_one_joint_tax_fiscal_household)
                ].NBFOYM.value_counts() #On a deux declarants 2, mais un seul foyer fiscal dans le logement, étrange TODO:


In [ ]:
##### To drop
id_more_than_one_joint_tax_fiscal_household

### couple_mar_pac_ss_enf

In [ ]:
fiscr_MM_OO = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_MM_OO)]
fiscr_MM_OO_ss_enf = fiscr_MM_OO[fiscr_MM_OO.NBPERSM == 2]
print(fiscr_MM_OO_ss_enf.NBFOYM.value_counts()) #Enlever mariés déclarations séparé
fiscr_MM_OO_ss_enf_1_foy = fiscr_MM_OO_ss_enf[fiscr_MM_OO_ss_enf.NBFOYM==1] #Enlever mariés déclarations séparé
df_couple_mar_pac_ss_enf = fiscr_MM_OO_ss_enf_1_foy

In [ ]:
fiscr_MM_OO = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_MM_OO)]
fiscr_MM_OO_ss_enf = fiscr_MM_OO[fiscr_MM_OO.NBPERSM == 2]
print(fiscr_MM_OO_ss_enf.NBFOYM.value_counts()) #Enlever mariés déclarations séparé
fiscr_MM_OO_ss_enf_1_foy = fiscr_MM_OO_ss_enf[fiscr_MM_OO_ss_enf.NBFOYM==1] #Enlever mariés déclarations séparé
df_couple_mar_pac_ss_enf = fiscr_MM_OO_ss_enf_1_foy

#### Fill structure in fiscr df

In [ ]:
for var in [ "Nb_child",
            "Nb_attached_child","Nb_attached_minor_child",
            "Nb_major_child","Nb_attached_major_child",
            "Nb_detatched_major_child"]:
    df_couple_mar_pac_ss_enf.loc[var] = 0

### couple_mar_pac_ac_enf sans majeurs

In [ ]:
fiscr_MMOO_ss_maj = fisc_revenu[((fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_MM_OO))
                          &(fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_enf_ss_majeurs_rataches)))]


In [ ]:
#Teste que la différence entre le nombre d'individus et le nombre d'enfants est de 2 (i.e. le couple)
print((fiscr_MMOO_ss_maj.NBPERSM - fiscr_MMOO_ss_maj.NBPAC).value_counts(normalize = True)[2.0] )
#0.034% de mauvaises obs, del
filter_2_parents_n_enf = (fiscr_MMOO_ss_maj.NBPERSM - fiscr_MMOO_ss_maj.NBPAC == 2)
fiscr_MMOO_ss_maj_match_n = fiscr_MMOO_ss_maj[filter_2_parents_n_enf]

#### Tests

In [ ]:
#Teste que le nombre de foyer fiscal est de 1
print(fiscr_MMOO_ss_maj_match_n.NBFOYM.value_counts(normalize = True))
#0.4% d'erreurs, pourquoi ?, del (TODO potentiel, couples qui se reforment et n'ont pas divorcés ?)
df_couple_mar_pac_ac_enf_ss_maj = fiscr_MMOO_ss_maj_match_n[fiscr_MMOO_ss_maj_match_n.NBFOYM == 1] 

#### Fill structure in fiscr df

In [ ]:
%%time
for var in [ "Nb_child","Nb_attached_child","Nb_attached_minor_child",]:
    df_couple_mar_pac_ac_enf_ss_maj[var] = df_couple_mar_pac_ac_enf_ss_maj.NBPAC

for var in [ "Nb_major_child","Nb_attached_major_child","Nb_detatched_major_child"]:
    df_couple_mar_pac_ac_enf_ss_maj[var] = 0

### couple_mar_pac_ac_enf ac majeurs tous rattachés

In [ ]:
fiscr_MMOO_ac_maj_rataches = fisc_revenu[((fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_MM_OO))
                          &(fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_majeurs_rataches)))]
#Teste que la différence entre le nombre d'individus et le nombre d'enfants est de 2 (i.e. le couple)
print((fiscr_MMOO_ac_maj_rataches.NBPERSM - fiscr_MMOO_ac_maj_rataches.NBPAC
      ).value_counts(normalize = True)[2.0] )

#0.034% de mauvaises obs, del
filter_2_parents_n_enf = (fiscr_MMOO_ac_maj_rataches.NBPERSM - fiscr_MMOO_ac_maj_rataches.NBPAC == 2)
fiscr_MMOO_ac_maj_rataches_match_n = fiscr_MMOO_ac_maj_rataches[filter_2_parents_n_enf]

#### Tests

In [ ]:
#Teste que le nombre de foyer fiscal est de 1
print(fiscr_MMOO_ac_maj_rataches.NBFOYM.value_counts(normalize = True))
#0.12% d'erreurs, pourquoi ?, del (TODO potentiel, couples qui se reforment et n'ont pas divorcés ?)
df_couple_mar_pac_ac_enf_ac_maj_ratach = fiscr_MMOO_ac_maj_rataches_match_n[fiscr_MMOO_ac_maj_rataches_match_n.NBFOYM == 1] 

In [ ]:
##TODO: tester que l'age est inférieur à 25 ans pour tout les rattachés.

### couple_mar_pac_ac_enf ac au moins un majeur non rattaché

In [ ]:
#info: NPPERSM - NBPAC == 2 + nombre de majeurs non ratachés

unique_declrant_2_grpby_CIDECI_O_or_M = fisc_revenu[
    fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_unique_declrant_2_grpby_CIDECI_O_or_M)
                                    ]

#Selection pour que différenced'individus et le nombre d'enfants est de 2 (i.e. le couple)
filter_more_than_2NBPERSM_NBPAC_diff = (unique_declrant_2_grpby_CIDECI_O_or_M.NBPERSM 
                    - unique_declrant_2_grpby_CIDECI_O_or_M.NBPAC)>2

two_foy_or_more_with_unique_2nd_declar =  unique_declrant_2_grpby_CIDECI_O_or_M[
                                                filter_more_than_2NBPERSM_NBPAC_diff]

nb_of_non_pac_not_couple_individual =(two_foy_or_more_with_unique_2nd_declar.NBPERSM 
                                      - two_foy_or_more_with_unique_2nd_declar.NBPAC)-2
two_foy_or_more_with_unique_2nd_declar[
    "nb_of_non_pac_not_couple_individual"]= nb_of_non_pac_not_couple_individual

#test NBFOYM == 1 + NB_maj_ratach    # On teste qu'il y a le foyer fiscal des parents et 1 foyer fiscal par maj rattaché
print((nb_of_non_pac_not_couple_individual 
       - (two_foy_or_more_with_unique_2nd_declar.NBFOYM-1)
      ).value_counts(normalize = True))
# #0.19% de mauvaises obs, del
### Make an assert

filter_nb_foy_is_nb_maj_non_ratach_plus_couple_foy = (nb_of_non_pac_not_couple_individual 
                                               == (two_foy_or_more_with_unique_2nd_declar.NBFOYM-1))
nb_foy_is_nb_maj_non_ratach_plus_couple_foy = two_foy_or_more_with_unique_2nd_declar[
                                                filter_nb_foy_is_nb_maj_non_ratach_plus_couple_foy]

In [ ]:
nb_foy_is_nb_maj_non_ratach_plus_couple_foy = nb_foy_is_nb_maj_non_ratach_plus_couple_foy.sort_values("ID_FISC_LOG_DIFF")

In [ ]:
nb_foy_is_nb_maj_non_ratach_plus_couple_foy.sort_values("ID_FISC_LOG_DIFF").head(5)

In [ ]:
two_foy_or_more_with_unique_2nd_declar.sort_values("ID_FISC_LOG_DIFF").head(5)

In [ ]:
nb_foy_is_nb_maj_non_ratach_plus_couple_foy.shape

#### Tests

##### Majeurs non ratach younger than their parents


- dans chaque logement fiscal, il y a un seul foyer fiscal composé d'un couple --> age des parents
- dans les autres logements fiscaux, il y a l'age des non ratachés vivant dans le logement.

Algorithme: 
Pour chaque logement fiscal vérfier : 
    min(age des parent) > max(age des non ratachés)

In [ ]:
(filter_nb_foy_is_nb_maj_non_ratach_plus_couple_foy).value_counts()

In [ ]:
nb_foy_is_nb_maj_non_ratach_plus_couple_foy.shape

In [ ]:
id_fisc_log_foy_is_nb_maj_non_ratach_plus_couple_foy = nb_foy_is_nb_maj_non_ratach_plus_couple_foy\
                                                        .ID_FISC_LOG_DIFF

In [ ]:
#To del if cell below working
filter_maj_non_ratach_plus_couple_foy = fisc_individu.ID_FISC_LOG_DIFF.isin(id_fisc_log_foy_is_nb_maj_non_ratach_plus_couple_foy)
fisci_non_ratach = fisc_individu[filter_maj_non_ratach_plus_couple_foy]

In [ ]:
### Bug ennorme de python ! comparer avec version du dessus
### Essayer de comprendre ?


# #To del if cell above working
# filter_maj_non_ratach_plus_couple_foy = fisc_individu.ID_FISC_LOG_DIFF.isin(id_fisc_log_foy_is_nb_maj_non_ratach_plus_couple_foy)
# fisci_non_ratach = fisc_individu[filter_maj_non_ratach_plus_couple_foy]



## Marche quand on renomme... plus de bug
# df = fisc_individu
# df["Key"] = df.ID_FISC_LOG_DIFF
# keys = id_fisc_log_foy_is_nb_maj_non_ratach_plus_couple_foy
# result = df[df.Key.isin(keys)].head(); result
# filter_keys = df.Key.isin(keys)
# result = df[filter_keys].head(); result

In [ ]:
assert len(fisci_non_ratach) > 10**6

In [ ]:
id_foy_couple = fisci_non_ratach.query("TYPE_FISC == '2'").ID_FISC_FOY_DIFF


In [ ]:
id_foy_couple_set = set(id_foy_couple.unique())

In [ ]:
id_foy_pers_non_ratach = (set(fisci_non_ratach.ID_FISC_FOY_DIFF.unique())) - id_foy_couple_set

In [ ]:
# teste que l'on a bien l'ensemble des foyers fiscaux dans les deux sets.
assert len(id_foy_couple_set)+len(id_foy_pers_non_ratach) == len(fisci_non_ratach.ID_FISC_FOY_DIFF.unique())

In [ ]:
#Ne prendre que les parents (et pas les enfants rattachés)
df_non_ratach_spouses = fisci_non_ratach.query("(TYPE_FISC=='1')|(TYPE_FISC=='2')")



youngest_spouse = df_non_ratach_spouses[
                            df_non_ratach_spouses.ID_FISC_FOY_DIFF.isin(id_foy_couple_set)
                        ].groupby("ID_FISC_LOG_DIFF")["ANAIS"].max() #prend le parent le plus jeunne

In [ ]:
oldest_detached = fisci_non_ratach[
                            fisci_non_ratach.ID_FISC_FOY_DIFF.isin(id_foy_pers_non_ratach)
                        ].groupby("ID_FISC_LOG_DIFF")["ANAIS"].min() #prend le parent le plus jeunne

In [ ]:
youngest_spouse.name = "youngest_spouse"
oldest_detached.name = "oldest_detached"
merge = pd.merge(youngest_spouse.to_frame(), oldest_detached.to_frame(), left_index=True,right_index=True)

In [ ]:
merge["Child_detached"] = (merge.youngest_spouse < merge.oldest_detached)
merge["Grand_parent_detached"] = (merge.youngest_spouse > merge.oldest_detached)

In [ ]:
id_log_grand_parent_detached = merge.query('Grand_parent_detached').index #On va laisser un peu tomber les familles avec un grand parent à charge.
id_log_child_detached = merge.query('Child_detached').index

In [ ]:
## TODO : trouver des tests ?

In [ ]:
fiscr_temp = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_log_child_detached)]

In [ ]:
fiscr_temp.TYPMEN9.value_counts()

In [ ]:
set_id_log_child_detached = set(id_log_child_detached.unique())
set_id_log_grand_parent_detached = set(id_log_grand_parent_detached.unique())

# Les célibataires (ou concubins)

### Celibataire_mar_pac_ss_enf

In [ ]:
fiscr_C = fisc_revenu[fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_C)]
#Test
fiscr_C_ss_enf = fiscr_C[fiscr_C.NBPERSM == 1]
print(fiscr_C_ss_enf.NBFOYM.value_counts()) #CHECK
fiscr_C_ss_enf_1_foy = fiscr_C_ss_enf[fiscr_C_ss_enf.NBFOYM==1] #CHECK
df_celib_ss_enf = fiscr_C_ss_enf_1_foy

### celib ac_enf sans majeurs

In [ ]:
fiscr_C_ss_maj = fisc_revenu[((fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_C))
                          &(fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_enf_ss_majeurs_rataches)))]

In [ ]:
#Teste que la différence entre le nombre d'individus et le nombre d'enfants est de 2 (i.e. le couple)
print((fiscr_C_ss_maj.NBPERSM - fiscr_C_ss_maj.NBPAC).value_counts(normalize = True)[1.0] )
#0.x% de mauvaises obs, del
filter_1_parent_n_enf = (fiscr_C_ss_maj.NBPERSM - fiscr_C_ss_maj.NBPAC == 1)
fiscr_C_ss_maj_rat_match_n = fiscr_C_ss_maj[filter_1_parent_n_enf]

#### Tests

In [ ]:
#Teste que le nombre de foyer fiscal est de 1
print(fiscr_C_ss_maj_rat_match_n.NBFOYM.value_counts(normalize = True))
##0 % d'erreur, tous les logements fiscaux avzec un seul déclarant "C" 
#n'ayant pas de majeurs dans le logement n'ont qu'un seul foyer fiscal

assert np.all(fiscr_C_ss_maj_rat_match_n.NBFOYM==1)

df_celib_ac_enf_ss_maj = fiscr_C_ss_maj_rat_match_n[fiscr_C_ss_maj_rat_match_n.NBFOYM == 1] 

In [ ]:
df_celib_ac_enf_ss_maj.TYPMEN9.value_counts() 

On a bien que des célib avec un enfant ou plus dans typmen9

In [ ]:
# id_log = df_couple_mar_pac_ac_enf_ss_maj.ID_FISC_LOG_DIFF   #Check visually
# fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(id_log)]

### single_ac_enf ac majeurs tous rattachés

In [ ]:
fiscr_C_ac_maj_rataches = fisc_revenu[((fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_declarants_C))
                          &(fisc_revenu.ID_FISC_LOG_DIFF.isin(id_fisc_log_majeurs_rataches)))]
#Teste que la différence entre le nombre d'individus et le nombre d'enfants est de 1 (i.e. le célibataire)
assert(np.all(fiscr_C_ac_maj_rataches.NBPERSM - fiscr_C_ac_maj_rataches.NBPAC == 1))




fiscr_C_ac_maj_rataches_match_n = fiscr_C_ac_maj_rataches

In [ ]:
#Teste que le nombre de foyer fiscal est de 1
print(fiscr_C_ac_maj_rataches_match_n.NBFOYM.value_counts(normalize = True))
df_single_ac_enf_ac_maj_ratach = fiscr_C_ac_maj_rataches_match_n[fiscr_C_ac_maj_rataches_match_n.NBFOYM == 1] 

In [ ]:
##Attention à la variable TYPMEN9, elle est fausse. 
df_single_ac_enf_ac_maj_ratach.TYPMEN9.value_counts()


In [ ]:
##TODO: tester que l'age est inférieur à 25 ans pour tout les rattachés.

In [ ]:
# id_log = df_single_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF   #Check visually
# fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(id_log)]

#### Fill structure in fiscr df

### celiataire_ac_enf ac au moins un majeur non rattaché

In [ ]:
#Impossible de savoir qui est parent, qui est enfant, qui est grand parent rattaché


#Règle: 
#0- Plusieurs foyers fiscaux.
#1- Un seul foyer avec  des personnes à charges --> ce foyer est parent célib
#2- Autres foyers tous déclanrants 1 sont "C" (Virer les veufs V, et divorcés D)

##Gérer les ages après
##4- SI moins de 18 ans d'écart d'age entre parents et autre individus, concubins --> on vire. 
##5- SI plus de 24 ans d'écart d'age entre parents et autre individus, grands parents --> on vire 

In [ ]:
to_drop_one_fiscal_household = fisc_revenu[fisc_revenu.NBFOYM==1].ID_FISC_LOG_DIFF
to_drop_maried = grpby_fisci[grpby_fisci.str.contains("M")].index
to_drop_pacs = grpby_fisci[grpby_fisci.str.contains("O")].index
to_drop_celib_ss_enf = df_celib_ss_enf.ID_FISC_LOG_DIFF
to_drop_celib_ss_maj_rat = df_single_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF


In [ ]:
# df = fisc_individu[~fisc_individu.ID_FISC_LOG_DIFF.isin(to_drop_more_than_one_fiscal_household)]

In [ ]:
# df.sort_values("ID_FISC_LOG_DIFF").head(100).groupby("ID_FISC_FOY_DIFF").agg(
#     {"AN_FISC":"count", "ID_FISC_LOG_DIFF":"first", "ANAIS":'min',"ANAIS":'max'}
#     ) .sort_values('ID_FISC_LOG_DIFF')

In [ ]:
df = fisc_individu[~fisc_individu.ID_FISC_LOG_DIFF.isin(to_drop_one_fiscal_household)]
df = df[~df.ID_FISC_LOG_DIFF.isin(to_drop_maried)]
df = df[~df.ID_FISC_LOG_DIFF.isin(to_drop_pacs)]
df = df[~df.ID_FISC_LOG_DIFF.isin(to_drop_celib_ss_enf)]
df = df[~df.ID_FISC_LOG_DIFF.isin(to_drop_celib_ss_maj_rat)].sort_values("ID_FISC_LOG_DIFF")

In [ ]:
#A ce stade il devrait nous rester les concubins, 
#le parent, avec son enfant non rataché ou le grand parent, et ses éventuels enfants rattachés.

In [ ]:
#Si moins de 18 ans  d'écart entre deux déclarants --> conjoints
#Si plus de 18 ans, d'écart entre plus vieux déclarant et autre déclarant --> enfant

## Note : les couples ayant plus de 18 ans d'écart sont rares, les individus ayant des enfants avant l'age de 18 sont rares.
## Clea devrait permettre d'avoir un taux d'erreur très faible.



##Cas limite, plus agé, deuxième plus agé et troisième plus agé (et n ème) ont moins de 18 ans d'écart ? 
##### On les vire.

In [ ]:
#Vieuw
df["ANAIS_min"] = df.ANAIS
df["ANAIS_max"] = df.ANAIS
df_grpby = df.groupby("ID_FISC_FOY_DIFF").agg(
            {"AN_FISC":"count", "ID_FISC_LOG_DIFF":"first",
             "ANAIS_min":'min', "ANAIS_max":'max',}
            ) .sort_values('ID_FISC_LOG_DIFF'); df_grpby.head()
df.drop(columns = ["ANAIS_min", "ANAIS_max"], inplace = True)
df_grpby.head()

##### Si moins de 18 ans  d'écart entre deux déclarants --> conjoints

In [ ]:
#Plus vieux déclarant par logement
oldest_in_log = df_grpby.reset_index().groupby("ID_FISC_LOG_DIFF").agg(
            { "ID_FISC_FOY_DIFF":"first",
             "ANAIS_min":'min'})

In [ ]:
df_oldest_in_log = oldest_in_log.rename(columns={"ANAIS_min": "ANAIS_Oldest_in_log"}).reset_index()

In [ ]:
df_oldest_in_log.head()

In [ ]:
%%time
merge = pd.merge(df_oldest_in_log, df,
         on= ["ID_FISC_LOG_DIFF"], how = "right")

In [ ]:
fisc_individu[fisc_individu.ID_FISC_LOG_DIFF == 2]

In [ ]:
merge = merge[merge.ANAIS_Oldest_in_log.notnull()]

In [ ]:
merge["Diff_with_oldest"] = (merge.ANAIS - merge.ANAIS_Oldest_in_log)

In [ ]:
merge[["ID_FISC_LOG_DIFF","ID_FISC_FOY_DIFF_y","ANAIS_Oldest_in_log","ANAIS","Diff_with_oldest" ]].head()

In [ ]:
merge["Diff_with_oldest_lower_than_18yo"] = merge.Diff_with_oldest <18 

In [ ]:
merge.groupby("ID_FISC_LOG_DIFF").Diff_with_oldest_lower_than_18yo.sum().value_counts()

In [ ]:
more_than_3 = merge.groupby("ID_FISC_LOG_DIFF").Diff_with_oldest_lower_than_18yo.sum()>=3

In [ ]:
merge[merge.ID_FISC_LOG_DIFF.isin(
    more_than_3[more_than_3 == True].index)][
    ["ID_FISC_LOG_DIFF","ID_FISC_FOY_DIFF_y",
     "ANAIS_Oldest_in_log","ANAIS","Diff_with_oldest" , "Diff_with_oldest_lower_than_18yo"]].head(20)

Les logements avec plus de 3 personnes qui sont proche de l'age du plus vieux du logement sont minoritaire.
Il s'agit souvent de plusieurs personnes très aggées, ou de familles recomposées ou l'écart d'age additionné entre le nouveau concubin et l'enfant du plus vieux adulte arrive à moins de 18 ans (eg. Mme qui a une fille de 18 ans se remet en couple avec Mr 9 ans de moins qu'elle). Ou de personne de même age (couple hébergeant un ami ? ect).

On va les ignorer.

In [ ]:
merge_drop_more_than_3 = merge[ ~ merge.ID_FISC_LOG_DIFF.isin(
                                            more_than_3[more_than_3 == True].index)]

##### On regarde maintenant que les enfants ont plus de 18 ans d'écart avec le plus deuxième plus vieux conjoint

In [ ]:
#%%time
#merge_drop_more_than_3.groupby("ID_FISC_LOG_DIFF").ANAIS.nsmallest(2)  #Trop long O(n^2) ?; on va y aller à la bourrin

In [ ]:
%%time
idx_min = merge_drop_more_than_3.groupby("ID_FISC_LOG_DIFF").ANAIS.idxmin()

In [ ]:
idx_min.values

In [ ]:
df_drop_oldest = merge_drop_more_than_3[~merge_drop_more_than_3.index.isin(idx_min.values)]

In [ ]:
grpby_second_oldest = df_drop_oldest.groupby("ID_FISC_LOG_DIFF").ANAIS.min()

In [ ]:
grpby_second_oldest.name = "ANAIS_Second_oldest"

In [ ]:
merge_with_scd_oldest = pd.merge(grpby_second_oldest.to_frame().reset_index(), merge_drop_more_than_3, 
         on = "ID_FISC_LOG_DIFF", how = "right")

In [ ]:
merge_with_scd_oldest[["ID_FISC_LOG_DIFF","ID_FISC_FOY_DIFF_y",
     "ANAIS_Oldest_in_log", "ANAIS_Second_oldest","ANAIS","Diff_with_oldest" , 
    "Diff_with_oldest_lower_than_18yo"]].tail(10)

In [ ]:
merge_with_scd_oldest["Diff_with_scd_oldest"] = (merge_with_scd_oldest.ANAIS - merge_with_scd_oldest.ANAIS_Oldest_in_log)

In [ ]:
(merge_with_scd_oldest.Diff_with_scd_oldest>=18).value_counts(normalize = True)

In [ ]:
merge_with_scd_oldest[(~(merge_with_scd_oldest.Diff_with_oldest<18))
                      &(merge_with_scd_oldest.Diff_with_scd_oldest>=18)].shape

In [ ]:
merge_with_scd_oldest["Parent"] = (merge_with_scd_oldest.Diff_with_oldest<18) #TODO:? gèrer les foyers fiscaux pour les concubins?

merge_with_scd_oldest["Child"] = ((~(merge_with_scd_oldest.Diff_with_oldest<18))
                      &(merge_with_scd_oldest.Diff_with_scd_oldest>=18))

##### Export child parent df 

In [ ]:
result_store_path = 'C:/Users/IMPTEMP_A_PACIFIC/Desktop/EDP_2016/Data/hdf/result.h5'

In [ ]:
merge_with_scd_oldest[["ID_FISC_FOY_DIFF_y","ORDREFIP","TYPE_FISC", "Child", "Parent", "ANAIS"]
                     ].convert_objects().to_hdf(result_store_path, "fisci_child_or_parent_{}".format(year))

In [ ]:
merge_with_scd_oldest.columns

##### Continue process

In [ ]:
(merge_with_scd_oldest.Parent).value_counts(normalize = True)

In [ ]:
(merge_with_scd_oldest["Child"]).value_counts(normalize = True)

In [ ]:
(merge_with_scd_oldest["Child"]|merge_with_scd_oldest["Parent"]).value_counts(normalize = False)
## 48 individus ne correspondent pas à cette définition --> on les vire

In [ ]:
condition = merge_with_scd_oldest["Child"]|merge_with_scd_oldest["Parent"]
id_to_drop = merge_with_scd_oldest[~condition].ID_FISC_LOG_DIFF

In [ ]:
merge_with_scd_oldest = merge_with_scd_oldest[~merge_with_scd_oldest.ID_FISC_LOG_DIFF.isin(id_to_drop)]

In [ ]:
merge_with_scd_oldest[
    merge_with_scd_oldest["Parent"]==True].groupby("ID_FISC_LOG_DIFF").size().value_counts()

293509 couples de concubins, 97749 célibataires avec enfants majeurs rattachés.  

In [ ]:
#Differentiate concubin from celib
number_of_parents = merge_with_scd_oldest[merge_with_scd_oldest["Parent"]==True].groupby("ID_FISC_LOG_DIFF").size()
id_log_concubin_with_or_without_children = number_of_parents[number_of_parents == 2].index
id_log_celib_with_detached_child = number_of_parents[number_of_parents == 1].index

In [ ]:
%%time
#Couple
couple_child_detached_or_attached_id_log = set(id_log_child_detached.unique())
couple_mar_pac_child_maj_ratach_id_log  = df_couple_mar_pac_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF
couple_mar_pac_child_ss_maj_id_log = df_couple_mar_pac_ac_enf_ss_maj.ID_FISC_LOG_DIFF
couple_mar_pac_ss_enf_id_log = df_couple_mar_pac_ss_enf.ID_FISC_LOG_DIFF




### Celibataire
id_log_celib_ss_enf=df_celib_ss_enf
id_log_celib_ac_enf_ss_maj=df_celib_ac_enf_ss_maj 
id_log_single_ac_single_ac_enf_ac_maj_ratach = df_single_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF
id_log_celib_with_detached_child

#Concubins 
id_log_concubin_with_or_without_children

In [ ]:
%%time
#Couple
couple_child_detached_or_attached_id_log =  set(id_log_child_detached.unique())
couple_mar_pac_child_maj_ratach_id_log  =  set(df_couple_mar_pac_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF.unique())
couple_mar_pac_child_ss_maj_id_log =  set(df_couple_mar_pac_ac_enf_ss_maj.ID_FISC_LOG_DIFF.unique())
couple_mar_pac_ss_enf_id_log =  set(df_couple_mar_pac_ss_enf.ID_FISC_LOG_DIFF.unique())
### Celibataire
id_log_celib_ss_enf = set(df_celib_ss_enf.ID_FISC_LOG_DIFF.unique())
id_log_celib_ac_enf_ss_maj =  set(df_celib_ac_enf_ss_maj .ID_FISC_LOG_DIFF.unique())
id_log_single_ac_single_ac_enf_ac_maj_ratach =  set(df_single_ac_enf_ac_maj_ratach.ID_FISC_LOG_DIFF.unique())
id_log_celib_with_detached_child =  set(id_log_celib_with_detached_child.unique())
#Concubins 
id_log_concubin_with_or_without_children =  set(id_log_concubin_with_or_without_children.unique())

In [ ]:
to_dump_id_log_dict = {"couple_child_detached_or_attached_id_fisc_log":couple_child_detached_or_attached_id_log,
                      "couple_mar_pac_child_maj_ratach_id_fisc_log":couple_mar_pac_child_maj_ratach_id_log,
                      "couple_mar_pac_child_ss_maj_id_fisc_log":couple_mar_pac_child_ss_maj_id_log,
                      "couple_mar_pac_ss_enf_id_fisc_log":couple_mar_pac_ss_enf_id_log,
                      "celib_ss_enf_id_fisc_log":id_log_celib_ss_enf,
                      "celib_ac_enf_ss_maj_id_fisc_log": id_log_celib_ac_enf_ss_maj,
                      "single_ac_single_ac_enf_ac_maj_ratach_id_fisc_log":id_log_single_ac_single_ac_enf_ac_maj_ratach,
                      "celib_with_detached_child_id_fisc_log":id_log_celib_with_detached_child,
                      "concubin_with_or_without_children_id_fisc_log":id_log_concubin_with_or_without_children
                      }




In [ ]:
for key, value in to_dump_id_log_dict.items():
        assert  isinstance(value,set)
        pickle.dump(value, 
                open(pickles_path+"/{}_{}.p".format(key, year), 'wb'))

### On verifie que les sets sont mutuellement exclusifs

In [ ]:
temp_set = set()
for set_name, set_data in to_dump_id_log_dict.items():
    print(set_name)
    print(temp_set.intersection(set_data))
    
    print(len(temp_set.union(set_data)))
    temp_set = temp_set.union(set_data)

In [ ]:
len(temp_set.union(set_data))/fisc_individu.ID_FISC_LOG_DIFF.nunique()

In [ ]:
complete_id_fisc_log_diff_set = set(fisc_individu.ID_FISC_LOG_DIFF.unique())

We are dropping 20% of the households.

In [ ]:
dropped_household_id_fisc_log_diff =  complete_id_fisc_log_diff_set - temp_set
len(dropped_household_id_fisc_log_diff)

In [ ]:
fisc_individu = fisc_individu.sort_values("ID_FISC_LOG_DIFF")

In [ ]:
### Il reste encore des choses à regler certains ménages devrraient se retrouver dans couples sans enfants, etc.

In [ ]:
fisc_individu[fisc_individu.ID_FISC_LOG_DIFF.isin(dropped_household_id_fisc_log_diff)]

In [ ]:
stop_time = datetime.datetime.now(); print("now: ",stop_time);
execution_time = stop_time - start_time; print("execution_time: ",execution_time)